# 数据聚合与分组计算

对数据集进行**分组**并对各组应用一个**函数**(无论是聚合还是转换),这是数据分析工作中的重要环节。在将数据集准备好之后,通常的任务就是**计算分组统计**或**生成透视表**。pandas提供了一个灵活高效的**gruopby**功能,它使你能以一种自然的方式对数据集进行**切片**、**切块**、**摘要**等操作。

关系型数据库和SQL(Structured Query Language,结构化查询语言)能够如此流行的原因之一就是其能够方便地对数据进行连接、过滤、转换和聚合。但是,像SQL这样的查询语言所能执行的分组运算的种类很有限。在本章中你将会看到,由于Python和pandas强大的表达能力,我们可以执行复杂得多的分组运算(利用任何可以接受pandas对象或NumPy数组的函数)。

在本章中,你将会学到:
* 根据一个或多个键(可以是函数、数组或DataFrame列名)拆分pandas对象。
* 计算分组摘要统计,如计数、平均值、标准差,或用户自定义函数。
* 对DataFrame的列应用各种各样的函数。
* 应用组内转换或其他运算,如规格化、线性回归、排名或选取子集等。
* 计算透视表或交叉表。
* 执行分位数分析以及其他分组分析。

In [3]:
import numpy as np

import pandas as pd
from pandas import Series
from pandas import DataFrame

import matplotlib.pyplot as plt
%matplotlib inline

## GroupBy技术

### 分组运算（split-apply-combine）

分组运算（split-apply-combine）：
* 拆分：
    * 通过一个或多个键对原数据进行拆分到不同组中；
* 应用：
    * 在不同组上应用函数计算得到结果；
* 合并：
    * 将结果合并到最终的结果对象中；

下图很好的展示了该过程：
![分组计算](https://github.com/NemoHoHaloAi/machine_learning/blob/master/python%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90/images/%E5%88%86%E7%BB%84%E8%AE%A1%E7%AE%97.png?raw=true)

### 分组键的可取情况

分组键可以有多种形式,且类型不必相同:
* 列表或数组,其长度与待分组的轴一样。
* 表示DataFrame某个列名的值。
* 字典或Series,给出待分组轴上的值与分组名之间的对应关系。
* 函数,用于处理轴索引或索引中的各个标签。
**注意**：后三种本质上是第一种的快捷方式，通过各种方式获取用于拆分对象的值，因此可以将这四种方式看做是如何获取用于拆分对象的值的四种方式即可，第一种是直接使用数组，第二种是取列名，第三种是映射关系，第四种是靠返回值；

注意：不管分组时表面上使用的是什么，最终都会转换成一个用于对应数据应该处于哪个分组的数组，数组上每个值，决定了相应位置的数据应该属于哪个分组；

### 分组示例

#### 使用Series做分组键 -- 例如df['key1']

In [4]:
df = DataFrame({'data1':[10,20,30,40,50],'data2':[40,50,60,70,80],
                'key1':['a','b','a','b','a'],'key2':['c','d','d','c','c']},
              index=['HL','LM','BL','JK','MP'])
df

,data1,data2,key1,key2
HL,10,40,a,c
LM,20,50,b,d
BL,30,60,a,d
JK,40,70,b,c
MP,50,80,a,c


In [6]:
# 对data1列数据按照key1分组并聚合计算平均值
df['data1'].groupby(df['key1']).mean() # 生成Series索引为key1的唯一值

key1
a    30
b    30
Name: data1, dtype: int64

In [8]:
# 对data1按照key1，key2分组并计算平均值
df['data1'].groupby([df['key1'],df['key2']]).mean() # 生成Series索引为key1，key2的唯一键组合

key1  key2
a     c       30
      d       30
b     c       40
      d       20
Name: data1, dtype: int64

#### 任意数组做分组键 -- 数组每个值对应同位置行的值，也就是强行有一种映射关系

In [9]:
arr = np.array(['aa','bb','cc','aa','cc'])
df['data1'].groupby(arr).mean()

aa    25
bb    20
cc    40
Name: data1, dtype: int64

#### 将列名(可以是字符串、数字或其他Python对象)用作分组键 -- 默认丢弃非数值组

In [14]:
df.groupby('key1').mean() # 使用列名作为分组键时不能针对某一列（Series）分组了就，因为Series没有该列名

,data1,data2
key1,,
a,30,60
b,30,60


#### groupby的size

In [17]:
df.groupby('key2').size()

key2
c    3
d    2
dtype: int64

### 对分组进行迭代

GroupBy对象支持迭代,可以产生一组二元元组(由分组名和数据块组成)。

#### 单键分组迭代

In [19]:
for name,group in df.groupby('key1'):
    print name
    print group
    print '\n'

a
    data1  data2 key1 key2
HL     10     40    a    c
BL     30     60    a    d
MP     50     80    a    c


b
    data1  data2 key1 key2
LM     20     50    b    d
JK     40     70    b    c




#### 多键分组迭代

In [20]:
for names,group in df.groupby(['key1','key2']):
    print names
    print group
    print '\n'

('a', 'c')
    data1  data2 key1 key2
HL     10     40    a    c
MP     50     80    a    c


('a', 'd')
    data1  data2 key1 key2
BL     30     60    a    d


('b', 'c')
    data1  data2 key1 key2
JK     40     70    b    c


('b', 'd')
    data1  data2 key1 key2
LM     20     50    b    d




#### 将分组结果转换为字典

In [25]:
group_dict = dict(list(df.groupby(['key1','key2'])))
for key in group_dict:
    print key
    print group_dict[key]
    print '\n'

('b', 'c')
    data1  data2 key1 key2
JK     40     70    b    c


('a', 'd')
    data1  data2 key1 key2
BL     30     60    a    d


('a', 'c')
    data1  data2 key1 key2
HL     10     40    a    c
MP     50     80    a    c


('b', 'd')
    data1  data2 key1 key2
LM     20     50    b    d




#### 在索引上分组 -- 指定axis=0

In [27]:
for name,group in df.groupby(['A','A','B','B','B'], axis=0):
    print name
    print group
    print '\n'

A
    data1  data2 key1 key2
HL     10     40    a    c
LM     20     50    b    d


B
    data1  data2 key1 key2
BL     30     60    a    d
JK     40     70    b    c
MP     50     80    a    c




### 选取一个或一组列 -- 可以直接对指定的列进行分组，或对分组结果取对应列

#### 对指定的列进行分组

In [36]:
df['data2'].groupby([df['key1'],df['key2']]).mean()

key1  key2
a     c       60
      d       60
b     c       70
      d       50
Name: data2, dtype: int64

#### 对分组结果取指定列 -- 这种方式是上一种方式的语法糖

In [29]:
df.groupby(['key1', 'key2'])['data2'].mean()

key1  key2
a     c       60
      d       60
b     c       70
      d       50
Name: data2, dtype: int64

#### 注意下述两种写法的不同之处

In [33]:
df.groupby(['key1'])['data2'].mean()

key1
a    60
b    60
Name: data2, dtype: int64

In [34]:
df.groupby(['key1'])[['data2']].mean()

,data2
key1,
a,60
b,60


比较：
* \['data2'\]：
    * 结果为Series；
    * Name属性为对应取的列名；
    * **DataFrame['列名']**得到的是对应列的**Series**形式；
* \[\['data2'\]\]：
    * 结果为DataFrame；
    * 索引为分组键，列为对应取的列名；
    * **DataFrame[['列名']]**得到的是对应列+原索引组成的**DataFrame**形式；

In [39]:
df['key1'] # 获取原索引+该列数据的Series

HL    a
LM    b
BL    a
JK    b
MP    a
Name: key1, dtype: object

In [40]:
df[['key1']] # 获取原索引+该列的DataFrame

,key1
HL,a
LM,b
BL,a
JK,b
MP,a


### 通过字典或Series进行分组

In [44]:
df = DataFrame(np.random.randn(5,5),
              columns=['a','b','c','d','e'],
              index=['01','02','03','04','05'])
df.ix[1:4,1:4] = np.nan # 设置几个nan值
df

/home/helong/anaconda2/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


,a,b,c,d,e
01,-0.982815,0.231855,1.035937,-0.747136,-0.541847
02,0.752139,NaN,NaN,NaN,1.961524
03,0.001247,NaN,NaN,NaN,3.538977
04,0.207718,NaN,NaN,NaN,-0.794822
05,-0.046359,0.636453,-1.048729,0.471528,-2.794262


#### 通过字典分组

In [53]:
dict_col = {'a':'A','b':'B','c':'A','d':'A','e':'B'}
df.groupby(dict_col, axis=1).mean()# 指定分组关系，等价于df.groupby(['A','B','A','A','B'], axis=1).mean()

,A,B
01,-0.231338,-0.154996
02,0.752139,1.961524
03,0.001247,3.538977
04,0.207718,-0.794822
05,-0.207853,-1.078905


#### 通过Series分组 -- 

In [58]:
series_col = Series({'a':'A','b':'B','c':'A','d':'A','e':'B'}) # 长度不一定要一致的
df.groupby(series_col, axis=1).mean()

,A,B
01,-0.231338,-0.154996
02,0.752139,1.961524
03,0.001247,3.538977
04,0.207718,-0.794822
05,-0.207853,-1.078905


### 通过函数分组

相较于字典或Series,Python函数在定义分组映射关系时可以更
有创意且更为抽象。任何被当做分组键的函数都会在各个索引值上被
调用一次,其返回值就会被用作分组名称。

#### 纯函数分组

In [68]:
df = DataFrame({'grade':[67,54,47,82,66]}, index=['Jack Jr.','Murphy','Mark Jr.','Lily','John Jr.'])
df.groupby(lambda name:'Jr.' in name).mean() # 根据名称中是否存在Jr.进行分组统计分数平均值

,grade
False,68
True,60


#### 函数混合其他分组 -- 先使用函数分为True，False两组，再根据数组继续细分

In [72]:
df.groupby([lambda name:'Jr.' in name,['1','2','2','1','2']]).mean() # 函数混合数组

grade
False 1   82.0
      2   54.0
True  1   67.0
      2   56.5

### 根据索引级别分组 -- 直接通过level参数指定分组级别即可

In [73]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],[1, 3, 5, 1, 3]], names=['cty', 'tenor'])
df = DataFrame(np.random.randn(4, 5), columns=columns)
df

cty          US                            JP          
tenor         1         3         5         1         3
0     -1.558237  0.827318  1.738935  0.613160 -0.240034
1      1.649638  0.111897 -2.122436 -1.191537  0.203578
2     -0.267392 -0.606987 -0.217272 -0.602829 -0.160219
3      1.020377 -0.859964  0.381138  0.253727  1.035110

In [78]:
df.groupby(level='cty', axis=1).mean() # 按照cty分组，也就是最外层索引

cty,JP,US
0,0.186563,0.336005
1,-0.493979,-0.120300
2,-0.381524,-0.363884
3,0.644418,0.180517


In [79]:
df.groupby(level=1, axis=1).mean() # 按照最内层索引分组

tenor,1,3,5
0,-0.472538,0.293642,1.738935
1,0.229051,0.157738,-2.122436
2,-0.435111,-0.383603,-0.217272
3,0.637052,0.087573,0.381138


## 数据聚合

## 分组级运算和转换

## 透视表和交叉表

## 示例：2012联邦